In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

from scipy.stats import entropy


import lib as lib


### Working on breast cancer data

In [2]:
# import some data to play with
#load the breast cancer dataset 
bcan = datasets.load_breast_cancer()
X = bcan.data
y = bcan.target

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.33, random_state=42)


# normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [3]:
# initialize params1 and params2

params1=[lib.initialize_weights(X_train.shape[1]) for i in range(100)]# a vector of shape 100,4
# call the initialize_weights function above

params2=[np.random.uniform() for i in range(100)]# a vector of shape 100
# use the np.random.uniform() function

# we have a list of 100 weight vectors (params1) and 100 thresholds (params2)
# convert them to array
params1=np.array(params1)
params2=np.array(params2)


print("Shape of params 1 (weights)",params1.shape)
print("Shape of params 2 (thresholds)",params2.shape)

Shape of params 1 (weights) (100, 30)
Shape of params 2 (thresholds) (100,)


In [4]:
z = lib.objective_fn_vector(params1, params2, X_train, y_train)
# Find the global minimum
param1_min = params1[z.argmin()] # use z.argmin()
param2_min = params2[z.argmin()] # use z.argmin()

print("param1_min",param1_min,"param2_min",param2_min)
print(lib.objective_fn(param1_min, param2_min, X_train, y_train))

param1_min [0.30071214 0.20861495 0.29497809 0.05005097 0.02996048 0.10968343
 0.72168448 0.94627581 0.46110004 0.38515513 0.34879775 0.88396629
 0.72522935 0.34172968 0.24960969 0.10241549 0.17898166 0.44724546
 0.34024543 0.06523101 0.11913937 0.25345845 0.68121686 0.20696321
 0.21434484 0.75753923 0.44909407 0.04698635 0.37761284 0.19663634] param2_min 0.9660295810274967
0.4441856282764507


In [5]:

# Hyper-parameter of the algorithm
c1 = c2 = 0.1
w1 = np.array([np.random.uniform() for i in range(X_train.shape[1])])
w2 = 0.8 
# Create particles
n_particles = 20
np.random.seed(100)
params1=[lib.initialize_weights(X_train.shape[1]) for i in range(n_particles)] # a vector of shape n_particles,4
# call the initialize_weights function above

params2=[np.random.uniform() for i in range(n_particles)]# a vector of shape n_particles
# use the np.random.uniform() function

params1=np.array(params1)
params2=np.array(params2)

print("params1 shape is ",params1.shape,"params2 shape is ",params2.shape)

params1 shape is  (20, 30) params2 shape is  (20,)


In [6]:
# define velocity of each weight of every particle
V_param1 = [lib.initialize_weights(X_train.shape[1])*0.1 for i in range(n_particles)] # shape is same as params1
# once again can use initialize_weights function

#define velocity of each threshold of every particle
V_param2 = np.array([np.random.uniform()*0.1 for i in range(n_particles)])# shape is same as params2
# once again use np.random.uniform() function

# Initialize objective values
pbest = (params1,params2)
pbest_obj = lib.objective_fn_vector(params1, params2, X_train, y_train)
gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()])
gbest_obj = pbest_obj.min()

print("pbest obj value for 20 particles are as follows",pbest_obj)
print("gbest obj value among all 20 particles is as follows",gbest_obj)
# note that gbest_obj should be the minimim of all pbest_obj

pbest obj value for 20 particles are as follows [0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461
 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461
 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461 0.6643461]
gbest obj value among all 20 particles is as follows 0.6643460978641622


In [7]:
def update(V_param1,V_param2, params1,params2, pbest, pbest_obj, gbest, gbest_obj):
    "Function to do one iteration of particle swarm optimization"
    # these have been already initialized in the previous cells
    
    # Update params
    r11,r12, r2 = np.random.rand(3)
    V_param1=w1*V_param1+c1*r11*(pbest[0] - params1)+ c2*r2*(gbest[0]-params1)
    V_param2=w2*V_param2+c1*r12*(pbest[1] - params2)+ c2*r2*(gbest[1]-params2)    
#     V = w * V + c1*r11*(pbest - params1) + c2*r2*(gbest.reshape(-1,1)-X)
    params1 = params1 + V_param1
    params2 = params2 + V_param2
    
    obj = lib.objective_fn_vector(params1, params2, X_train, y_train)
    for i in range(pbest[0].shape[0]):
        if pbest_obj[i]>=obj[i]:
            
            pbest[0][i]=params1[i] # update pbest[0][i] with value of params1[i]
            pbest[1][i]=params2[i] # update pbest[1][i] 
            pbest_obj[i]=obj[i]    # also update pbest_obj[i]

            
    gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()]) # update gbest to contain the best from params1 and params 2
    gbest_obj = pbest_obj.min() # update gbest to get the minimum of pbest_obj
 


In [8]:
for i in range(100):
    update(V_param1,V_param2, params1,params2, pbest, pbest_obj, gbest, gbest_obj)
print("PSO found best solution at f({})={}".format(gbest, gbest_obj))
print("Global optimal at f({})={}".format([param1_min,param2_min], lib.objective_fn(param1_min, param2_min, X_train, y_train)))


PSO found best solution at f((array([3.00936778, 1.12660482, 1.56050399, 7.23188626, 0.77120723,
       0.48653109, 2.09906857, 8.46237113, 1.3520507 , 2.71894176,
       2.02967019, 3.19420539, 0.51400385, 1.93887359, 5.28577265,
       2.62355492, 6.93243905, 3.94522167, 9.77298962, 2.7123689 ,
       6.19379182, 3.98661795, 1.16969601, 0.36742681, 1.97200725,
       7.52395933, 4.94227845, 5.30726597, 4.33239623, 0.67707827]), 0.4098464718143954))=0.6643460978641622
Global optimal at f([array([0.30071214, 0.20861495, 0.29497809, 0.05005097, 0.02996048,
       0.10968343, 0.72168448, 0.94627581, 0.46110004, 0.38515513,
       0.34879775, 0.88396629, 0.72522935, 0.34172968, 0.24960969,
       0.10241549, 0.17898166, 0.44724546, 0.34024543, 0.06523101,
       0.11913937, 0.25345845, 0.68121686, 0.20696321, 0.21434484,
       0.75753923, 0.44909407, 0.04698635, 0.37761284, 0.19663634]), 0.9660295810274967])=0.4441856282764507


In [9]:
def cal_information_gain(labels, split):
    num_samples = len(labels)
    num_split_1 = sum(split)
    num_split_2 = num_samples - num_split_1
    prob_split_1 = num_split_1 / num_samples
    prob_split_2 = 1 - prob_split_1
    
    entropy_before = entropy(labels)
    
    if num_split_1 == 0 or num_split_2 == 0:
        return 0
    
    labels_split_1 = labels[split]
    labels_split_2 = labels[~split]
    
    entropy_split_1 = entropy(labels_split_1)
    entropy_split_2 = entropy(labels_split_2)
    
    info_gain = entropy_before - prob_split_1 * entropy_split_1 - prob_split_2 * entropy_split_2
    return info_gain

def cal_entropy(labels):
    num_samples = len(labels)
    if num_samples == 0:
        return 0
    prob_1 = sum(labels) / num_samples
    prob_2 = 1 - prob_1
    entropy = -prob_1 * np.log2(prob_1 + 1e-10) - prob_2 * np.log2(prob_2 + 1e-10)
    return entropy


In [10]:
max_tree_size=128
all_optimized_weights_list=[None for i in range(max_tree_size)]
all_optimized_thresh_list=[None for i in range(max_tree_size)]
all_dataset_sizes_list=[None for i in range(max_tree_size)]
all_IG_list=[None for i in range(max_tree_size)]


In [13]:
   # I just commented this line #np.random.seed(100) and now we can have different thrshold 
    
def find_best_params(train_x,train_y,test_x,test_y,node_number):
    '''
    recursive function to get the best set of weights
    '''
    print("node_number",node_number,"data shape",train_x.shape)
    # exit condition 1: if the node_number is more than the maximum tree size, return
    if node_number>=max_tree_size:
        return
    # exit condition 2: if the training dataset has one or less rows, return 
    if train_x.shape[0]<=1:
        return 
    # exit condition 3: if the train_y has values from only one class (only 0s or only 1s and so on)
    if len(set(train_y))==1:
        return

    # use the initialized lists as global
    global all_optimized_weights_list
    global all_optimized_thresh_list
    global all_dataset_sizes_list
    global all_IG_list

    # Hyper-parameter of the algorithm
    c1 = c2 = 0.1
    w1 = np.array([np.random.uniform() for i in range(train_x.shape[1])])
    w2 = 0.8 

    # Create particles
    n_particles = 20
    #np.random.seed(100)
    params1=[lib.initialize_weights(train_x.shape[1]) for i in range(n_particles)]
    params2=[np.random.uniform() for i in range(n_particles)]
    params1=np.array(params1)
    params2=np.array(params2)

    # define velocity of each weight of every particle
    V_param1 = [lib.initialize_weights(train_x.shape[1])*0.1 for i in range(n_particles)]

    # define velocity of each threshold of every particle
    V_param2 = np.array([np.random.uniform()*0.1 for i in range(n_particles)])

    # Initialize objective values
    pbest = (params1,params2)
    pbest_obj = lib.objective_fn_vector(params1, params2, train_x, train_y)
    gbest=(params1[pbest_obj.argmin()],params2[pbest_obj.argmin()])
    gbest_obj = pbest_obj.min()

    for i in range(100):
        update(V_param1,V_param2, params1,params2, pbest, pbest_obj, gbest, gbest_obj)
    
    # add the achieved optimized values to the lists
    all_optimized_weights_list[node_number]=gbest[0]
    all_optimized_thresh_list[node_number]=gbest[1]
    all_dataset_sizes_list[node_number]=train_x.shape[0]
    all_IG_list[node_number]=cal_information_gain(train_y,train_y>=gbest[1])
    
    new_ys=np.dot(train_x,gbest[0])
    
    # normalize the new_ys
    new_ys=(new_ys-np.min(new_ys))/(np.max(new_ys)-np.min(new_ys))
    
    # chop the data into two parts: left
    train_x_left=train_x[new_ys>=gbest[1]]
    train_y_left=train_y[new_ys>=gbest[1]]
    left_child_node_num=node_number*2+1
    
    # chop the data into two parts: right
    train_x_right=train_x[new_ys<gbest[1]]
    train_y_right=train_y[new_ys<gbest[1]]
    right_child_node_num=node_number*2+2
    
    # exit condition 4: return if information gain is 0
    if all(ig == 0 for ig in all_IG_list):
        return
    
    print("Left",train_x_left.shape)
    print("Right",train_x_right.shape)
    # make the recursion call for left
    find_best_params(train_x_left,train_y_left,test_x,test_y,left_child_node_num)
    # make the recursion call for right
    find_best_params(train_x_right,train_y_right,test_x,test_y,right_child_node_num)    
    
    

In [14]:
node_number=0
find_best_params(X_train,y_train,X_test,y_test,node_number)

node_number 0 data shape (381, 30)
Left (135, 30)
Right (246, 30)
node_number 1 data shape (135, 30)
Left (3, 30)
Right (132, 30)
node_number 3 data shape (3, 30)
node_number 4 data shape (132, 30)
Left (131, 30)
Right (1, 30)
node_number 9 data shape (131, 30)
Left (11, 30)
Right (120, 30)
node_number 19 data shape (11, 30)
node_number 20 data shape (120, 30)
Left (11, 30)
Right (109, 30)
node_number 41 data shape (11, 30)
Left (1, 30)
Right (10, 30)
node_number 83 data shape (1, 30)
node_number 84 data shape (10, 30)
Left (2, 30)
Right (8, 30)
node_number 169 data shape (2, 30)
node_number 170 data shape (8, 30)
node_number 42 data shape (109, 30)
Left (50, 30)
Right (59, 30)
node_number 85 data shape (50, 30)
Left (7, 30)
Right (43, 30)
node_number 171 data shape (7, 30)
node_number 172 data shape (43, 30)
node_number 86 data shape (59, 30)
Left (5, 30)
Right (54, 30)
node_number 173 data shape (5, 30)
node_number 174 data shape (54, 30)
node_number 10 data shape (1, 30)
node_number

In [15]:
X_train.shape

(381, 30)

In [16]:
all_optimized_thresh_list

[0.3625126367788257,
 0.8797227016832677,
 0.8054404182500315,
 0.4098464718143954,
 0.018922415589424868,
 0.3646608762192656,
 0.6577243295649515,
 None,
 0.4098464718143954,
 0.7086799113230741,
 0.4098464718143954,
 0.41849359741967007,
 0.4098464718143954,
 0.6874571439988902,
 0.5603088062139597,
 None,
 None,
 None,
 0.4098464718143954,
 None,
 0.6905962587057257,
 None,
 0.4098464718143954,
 0.9376587928430847,
 0.6502357161557439,
 0.4098464718143954,
 0.4098464718143954,
 0.8621416786588048,
 0.9872218440931296,
 0.12054653837101204,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 0.4098464718143954,
 None,
 None,
 0.7050223997360379,
 0.5370619249524076,
 None,
 None,
 0.4098464718143954,
 0.4098464718143954,
 0.9077105475715062,
 0.4098464718143954,
 0.4098464718143954,
 0.28739690773826754,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 0.4098464718143954,
 None,
 0.9527895441536234,
 None,
 0.4806752234835754,
 0.2909177010286963,
 None,
 None,
 

In [47]:
thresh=all_optimized_thresh_list[0]